# Imports

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from rake_nltk import Rake
import re

# 1. Load Data

In [4]:
df = pd.read_csv('..\\tweets.csv', header=0)

# 2. Pre-Processing
remove unnecessary stuff

## 2.1 Remove:
- Numbers
- Words of Length <= 2
- non alphabet chars except #

In [5]:
PROBLEM_CHARS = re.compile(r'[^\w#\s]')
def clean(w):
    return re.sub(PROBLEM_CHARS, '', w)

In [6]:
data = df.text.apply(lambda t: " ".join([clean(w) for w in t.split()]))

In [7]:
numeric = re.compile(r'\w*\d+\w*')
def accepted_word(w):
    return len(w) > 2 and re.search(numeric, w) is None

In [8]:
data = data.apply(lambda t: " ".join([w for w in t.split() if accepted_word(w)]))

In [9]:
def accepted(t):
    words = t.split()
    for w in words:
        if not accepted_word(w):
            return 'not accepted'
    return 'accepted'

data.apply(lambda t: accepted(t)).value_counts()

accepted    45071
Name: text, dtype: int64

# 3. TF-IDF
A pre TF-IDF to get the first batch of frequent topics and filter them later by mining their stopwords

In [10]:
# TF-IDF Transformer
tf_Idf = TfidfVectorizer()

# Transform & Fit Data
tf_Idf_fit = tf_Idf.fit_transform(data)

# Get Features (Columns)
feature_names = tf_Idf.get_feature_names()

In [11]:
len(feature_names)

106365

In [12]:
words = pd.Series(feature_names)

# 4. Mine Stopwords

In [13]:
stopwords = []

## 4.1 Words of length = 2 are normally unnecessary

In [14]:
_2_char_words = words[words.str.len() == 2]
_2_char_words.count()

115

In [15]:
for w in _2_char_words:
    print(w)

__
ad
ai
at
ba
be
by
ca
cd
cj
ct
di
dr
ep
er
eu
ff
fl
fm
gb
gi
go
gp
hi
hr
in
is
it
jo
ki
ld
lr
ma
md
me
mo
mt
nc
ng
nh
ny
oh
om
on
or
ot
pa
pk
pm
pr
pt
ri
rn
rt
sc
se
si
so
st
tb
tn
to
tv
tw
ua
uk
un
us
va
vc
we
ze
آت
ال
اي
بس
بـ
به
حب
حج
حر
حظ
حق
حل
رب
شر
شه
شو
شي
صح
طب
طز
عا
غد
فل
فن
في
قم
لأ
لا
لـ
لل
له
ما
مح
مخ
مع
من
مي
هل
هم
هو
هي
يا
番茄


In [16]:
stopwords += list(_2_char_words)
len(stopwords)

115

## 4.2 Check words of length = 3  

In [17]:
_3_char_words = words[words.str.len() == 3]
_3_char_words.count()

4028

In [18]:
for w in _3_char_words:
    print(w)

_عم
_هل
aal
aam
aan
abc
abd
abe
abi
abl
abp
abt
abu
aca
acc
ach
aci
acm
aco
acp
acs
act
add
ade
ads
adı
aec
aed
afo
afp
afs
aft
age
ago
aha
aid
aim
ain
air
aja
aje
aka
akh
akl
ako
ale
alf
ali
all
alo
als
alt
ama
amb
amd
ami
amo
amp
amr
ana
and
ang
ano
ans
anu
anw
any
aon
aot
apa
app
apr
apt
arc
are
ark
arm
art
arw
arz
asa
asc
ash
ask
ass
así
atc
atm
atv
atx
aub
auc
aue
auf
aug
auh
auj
aus
aux
avd
ave
awe
axe
aya
aye
baa
bab
bac
bad
bae
bag
ban
bar
bas
bat
bau
bay
bbc
bbq
bcg
bcp
bcs
bcz
bda
bde
bdi
bdl
bds
beb
bec
bed
beg
bei
bek
bel
ben
bet
bey
bge
bhe
bhw
bid
big
bil
bin
bio
bir
bit
biz
bkl
bla
blc
blg
blm
bln
bls
blu
bmc
bmi
bmj
bmt
bob
boo
bot
bou
bow
box
boy
bpa
bpi
bra
brb
bro
brs
bsa
bsn
bss
btc
bts
btw
bug
bus
but
buy
bye
c_h
caf
cam
can
cap
car
cas
cat
cbd
cbt
ccm
ccp
ccs
cct
ccu
cda
cdc
cdl
cdn
cds
cee
ceo
ces
cet
cfr
cgi
chi
chp
chs
chu
cma
cmc
cme
cmh
cmo
cna
cnn
cnw
cob
cod
col
com
con
coo
cos
cou
cov
cow
cox
coz
cri
cry
csc
cso
csp
csr
cst
cte
ctr
cum
cup
cut
cuz
cvs
cya


دام
دان
داي
دبة
دبت
دبر
دبس
دبي
دجل
دحة
دخل
دخن
درب
درس
درك
دعا
دعت
دعس
دعم
دعي
دفع
دفن
دفي
دقة
دقت
دقق
دقن
دقه
دقو
دلع
دلو
دما
دمت
دمج
دمر
دمع
دمك
دمه
دمى
دنب
دنم
دني
دهب
دهن
دوا
دوب
دود
دور
دوز
دول
دوم
دون
ديب
دير
ديل
دين
ديو
ذئب
ذاب
ذات
ذاق
ذاك
ذبح
ذرة
ذعر
ذقن
ذكر
ذكى
ذكي
ذلا
ذلة
ذلك
ذمة
ذمت
ذنب
ذهب
ذوق
ذوي
ذيل
رأس
رأى
رأي
رئة
رئس
راح
راد
راس
رام
راي
ربا
ربة
ربت
ربح
ربط
ربع
ربك
ربن
ربه
ربو
ربى
ربي
رتب
رتش
رجآ
رجع
رجل
رحب
رحت
رحل
رحم
رخص
ردا
ردة
ردت
رده
ردو
ردي
ررب
رزق
رسم
رشا
رشة
رشح
رصد
رضا
رضع
رضى
رضي
رطل
رعب
رعد
رغم
رفض
رفع
رقص
رقم
ركب
ركز
ركض
ركع
ركن
رلى
رمت
رمز
رمق
رمى
رمي
رنا
رند
رهن
روج
روح
روس
روق
رول
روم
روي
ريب
ريت
ريح
رير
ريس
ريع
ريم
رين
ريو
رہا
زات
زاد
زار
زال
زبط
زجه
زحط
زحف
زحل
زخف
زخم
زده
زرت
زرع
زعل
زفة
زفت
زفر
زقف
زقن
زلة
زلت
زلم
زمر
زمط
زمن
زنا
زهر
زهق
زوج
زور
زيت
زيد
زين
سأل
سؤل
سئل
سات
ساح
ساد
سار
ساز
سام
سان
ساو
سبأ
سبب
سبت
سبع
سبق
سبل
سبه
ستة
ستر
ستك
ستي
سجل
سجن
سحب
سحر
سحق
سحل
سخر
سدا
سدد
سدو
سدى
سدي
سرآ
سرا
سرب
سرف
سرق
سره
سري
سطب
سطح
سطل
سطو
سعة
سعد
سعر
سعل


## *Observation*:
### `Some words are useful while most are not`

# 5. Add external stopwords

In [20]:
with open('../../stop-words/arabic.txt', 'r', encoding='utf-8') as f:
    stopwords += f.read().split('\n')

with open('../../stop-words/english.txt', 'r', encoding='utf-8') as f:
    stopwords += f.read().split('\n')

# 6. Group Data

In [21]:
df.text = data
mo7afazat = df.groupby('MOHAFAZA_AR')

# 7. Apply TF-IDF & RAKE with the stopwords

In [22]:
ngram = (3, 3)
r = Rake(max_length=ngram[1], stopwords=stopwords)

for name_ar, mo7afaza_df in mo7afazat:    
    # TF-IDF Transformer
    tf_Idf = TfidfVectorizer(stop_words=stopwords, ngram_range=ngram)

    # Transform & Fit Data
    tf_Idf_fit = tf_Idf.fit_transform(mo7afaza_df.text)

    # Get Features (Columns)
    feature_names = tf_Idf.get_feature_names()

    # # Transform the Sparse Matrix
    dense = tf_Idf_fit.todense()
    denselist = dense.tolist()
    
    # RAKE algorithm
    r.extract_keywords_from_text(" ".join(mo7afaza_df.text))
    rake_res = r.get_ranked_phrases_with_scores()
    
    break

d:\desktop\github\m2-project\twittervenv\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [23]:
len(feature_names)

16609

In [24]:
tf_idf_table = pd.DataFrame(denselist, columns=feature_names)
tf_idf_table

,aal mesrashfa fiye,aal mestashfa iza,aalam iran lebnen,aam addem hejra,aan need season,aana hejje jdide,aaronrichterman ericmeyerowitz useful,abbot specialist proximie,abdullah cant waysjordan,abdullah rabeeah king,...,يوما يوما يقترب,يومها بسنة اغتيالات,يومهم أرخص فاتورة,يوميا لتحصل قوتها,يومين حبلش بحالي,ڤايروس كورونا fadifakhryalame,ڤايروس كورونا يلتزم,ڤيروس الكورونا أفيدونا,ڤيروس كورونا السبب,ڤيروس كورونا يأخذ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
tf_idf_result = pd.DataFrame()
tf_idf_result[['topic', 'score']] = tf_idf_table.mean().reset_index()
tf_idf_result = tf_idf_result.sort_values('score', ascending=False).reset_index(drop=True)
tf_idf_result

,topic,score
0,تدفعنا للقول الحكومة,0.003022
1,نحن أمام واقع,0.003022
2,خطير والموضوعية تدفعنا,0.003022
3,واقع خطير والموضوعية,0.003022
4,والموضوعية تدفعنا للقول,0.003022
...,...,...
16604,لبنان تسجل إصابة,0.000118
16605,الدفعة مساعدات المملكة,0.000118
16606,الفلسطينية لمواجهة فيروس,0.000118
16607,لمواجهة فيروس كورونا,0.000118


In [26]:
rake_result = pd.DataFrame(rake_res, columns=['score', 'topic'])
rake_result

,score,topic
0,9.0,يوجهون الإتهام المباشر
1,9.0,يكرم المرء يهان
2,9.0,يتراجع عنصر الشباب
3,9.0,ووهان تشرين الأول
4,9.0,هيك رسائل الواتساب
...,...,...
2135,1.0,adilla
2136,1.0,accreditation
2137,1.0,abiding
2138,1.0,abbott


# 8. Combine both results

In [27]:
tf_idf_data = tf_idf_result.topic[:100]
rake_data = rake_result.topic[:100]

In [28]:
for t in tf_idf_data:
    print(t)

تدفعنا للقول الحكومة
نحن أمام واقع
خطير والموضوعية تدفعنا
واقع خطير والموضوعية
والموضوعية تدفعنا للقول
للقول الحكومة الجديدة
أمام واقع خطير
والمؤشرات كورونا لبنان_ينتقض
لبنان_ينتقض رفيق_الحريري بهاء_الحريري
كورونا لبنان_ينتقض رفيق_الحريري
الجديدة تتحمل مسؤولية
فشله والمؤشرات كورونا
ستتحمل حتما مسؤولية
أثبت فشله والمؤشرات
سابقا أثبت فشله
حتما مسؤولية تبنيها
وصلت إليه البلاد
تبنيها نهجا سابقا
إليه البلاد لكنها
مسؤولية تبنيها نهجا
البلاد لكنها ستتحمل
مسؤولية وصلت إليه
الحكومة الجديدة تتحمل
تتحمل مسؤولية وصلت
نهجا سابقا أثبت
لكنها ستتحمل حتما
beirutcommag via beirutcommag
أكثر الأخبار متابعة
شاهد أكثر الأخبار
reminder stayhome كورونا_لبنان
إصابة جديدة بكورونا
taking break corona
break corona covid
corona covid كورونا_لبنان
earth taking break
حالة كورونا لبنان
اصابة كورونا لبنان
كورونا لبنان ماذا
hands wash hands
wash hands wash
رفيق_الحريري بهاء_الحريري سعد_الحريري
كورونا لبنان لبنان_ينتفض
everyones health safety
لازملنا اجراءات صرامي
خليك_بالبيت كورونا لبنان_ينتفض
coz youre hot
كورونا_لبن

In [29]:
for t in rake_data:
    print(t)

يوجهون الإتهام المباشر
يكرم المرء يهان
يتراجع عنصر الشباب
ووهان تشرين الأول
هيك رسائل الواتساب
هلق مجبورين عالحجر
نطاقنا البلدي ونقله
منيح لوين هالمخاطرة
منظمة_الصحة_العالمية تحذر تحول
مقيدا بقيود صدئة
مركز_الملك_سلمان_للإغاثة يسلم الدفعة
مرضىمرض مرض بيفرق
مرجحة انت تصل
محمد_بن_زايد اتصال هاتفي
لمس العملة الورقية
لكان الوضع افضل
لبنان_كورونا smiley explaining
كورونا_في_لبنان نقول للصحافي
قانون_على_المعتر برج خليفة
فيروس_الفساد الديتول يقضي
فوائد الجراد للبشرة
فهم ونشر الوعي
فعلو زمن الحروب
عمدة_نيويورك خير دليل
طلاب_الجامعة_اللبنانية لعبة أوقفوا
شعبوي يلقى استحسان
سوق_الأسهم وهبوط الفائدة
سبقته عشرات الفيروسات
ساعة لماذالانو شعب
رئيس بهل وقت
دلعونا وعلى دلعونا
درهم لاحتواء تداعيات
حياتها الطبيعيةاستهتار واضح
حياتنا جميعا والخطوات
حول_العالم يضع كمامته
حشيش تيجى خاص
تعلم لهلق شغلي
تسجيل حالتين إيجابيتين
تحرير محضر بحقه
تحركات الحكومة جزء
تحدي_الخير وتتحدى مشاهير
بؤرة لتفشي الوباء
اوصلهم للجوع يرحمون
اهلكن كلن فوق
الوسائل الإعلامية والناس
الممارسات نكسة للإجراءات
المستشفى_اللبناني_الكندي